In [2]:
import pandas as pd

In [7]:
test_df = pd.read_csv("test.csv")

In [9]:
test_df.head()

,key,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,08:24.0,2015-01-27 13:08:24 UTC,-73.973320,40.763805,-73.981430,40.743835,1
1,08:24.0,2015-01-27 13:08:24 UTC,-73.986862,40.719383,-73.998886,40.739201,1
2,53:44.0,2011-10-08 11:53:44 UTC,-73.982524,40.751260,-73.979654,40.746139,1
3,12:12.0,2012-12-01 21:12:12 UTC,-73.981160,40.767807,-73.990448,40.751635,1
4,12:12.0,2012-12-01 21:12:12 UTC,-73.966046,40.789775,-73.988565,40.744427,1


날짜 형변환

In [10]:
test_df['time'] = pd.to_datetime(test_df.pickup_datetime, format='%Y-%m-%d %H:%M:%S UTC')

In [11]:
# 날짜 추출하기
test_df['log_date'] = test_df['time'].dt.date
 
# 시간 추출하기
test_df['log_time'] = test_df['time'].dt.time
 
# 시 추출하기
test_df['log_hour'] = test_df['time'].dt.hour
 
# 분 추출하기
test_df['log_min'] = test_df['time'].dt.minute
# 연도 추출하기
test_df['log_year'] = test_df['time'].dt.year

# 월 추출하기
test_df['log_month'] = test_df['time'].dt.month

# 일 추출하기
test_df['log_day'] = test_df['time'].dt.day

##### 시간 변수 그룹화

In [13]:
def is_7to15(x):
    if (x >= 7) & (x <= 15):
        return 1
    else:
        return 0

In [15]:
test_df['7to15'] = test_df['log_hour'].apply(is_7to15)

In [16]:
def is_16to19(x):
    if (x >= 16) & (x < 20):
        return 1
    else:
        return 0

In [17]:
test_df['16to19'] = test_df['log_hour'].apply(is_16to19)

In [18]:
def is_20to6(x):
    if (x >= 20) | (x <= 6):
        return 1
    else:
        return 0

In [19]:
test_df['20to6'] = test_df['log_hour'].apply(is_20to6)

##### 요일 추출 -> weekday 변수 생성

In [20]:
# 요일[1,7]. 월요일(1) ~ 일요일(7)

test_df['dayofweek'] = test_df['time'].dt.strftime("%u")

In [21]:
def is_weekday(x):
    if (int(x) < 6):
        return 1
    else:
        return 0

In [22]:
test_df['weekday'] = test_df['dayofweek'].apply(is_weekday)

#### 추가 요금 여부 : is_extrachange

In [24]:
test_df['is_extracharge'] = 0

In [28]:
# 20시부터 6시까지

test_df.loc[test_df['20to6'] == 1, 'is_extracharge'] = 1

In [29]:
# 월~금 16시부터 20시까지

test_df.loc[(test_df['weekday'] == 1) & (test_df['16to19'] == 1), 'is_extracharge'] = 1

In [30]:
pickup = test_df[['pickup_longitude','pickup_latitude']]
pd.DataFrame(pickup)

dropoff = test_df[['dropoff_longitude','dropoff_latitude']]
pd.DataFrame(dropoff)

,dropoff_longitude,dropoff_latitude
0,-73.981430,40.743835
1,-73.998886,40.739201
2,-73.979654,40.746139
3,-73.990448,40.751635
4,-73.988565,40.744427
...,...,...
9909,-73.955643,40.780388
9910,-73.960213,40.776371
9911,-73.789742,40.647011
9912,-73.939178,40.801731


In [31]:
distance = []

In [32]:
from geopy.distance import distance

In [33]:
dist = []
for i in range(len(dropoff.index)):
    result = distance(pickup.values[i], dropoff.values[i]).m
    dist.append(result)
print(dist)

[1094.5927051339195, 1474.196641480398, 357.0756761812017, 1150.115807142539, 2875.8033858122208, 2177.4496191316953, 1186.193330391389, 23453.883278060377, 2333.779921916147, 1249.6422360929164, 1259.1664638231691, 5113.554027005278, 681.4082207898864, 2213.8219184422483, 1207.6641372903466, 3810.534083076371, 386.3783568500437, 2735.583402769968, 268.20766583800514, 562.2381689359522, 1770.4953457767242, 2400.98350206299, 539.1869502976039, 1412.0582102539154, 3192.4286341265038, 1279.0362025685258, 14616.630513683034, 6430.554752209208, 517.5042161158525, 3971.6276514587385, 1053.1911204607075, 686.534857384554, 309.2690415422961, 1046.5274729189553, 1061.2882230328757, 1352.5365847926641, 405.2447532414453, 1190.407491609927, 2077.053497653273, 325.8441370414716, 4850.07695579766, 2259.6788070421144, 4667.7405396517825, 967.4557555192079, 412.27000661094684, 2103.993960160897, 1166.3944011075635, 1605.0336194972444, 1324.3024485524231, 821.8428747018676, 3728.0798103748816, 3837.08

In [35]:
test_df['dist'] = dist

In [38]:
test_df = test_df.drop(columns=['key','time','pickup_datetime','pickup_latitude','dropoff_longitude','dropoff_latitude','pickup_longitude','log_date','log_time','log_date'])

In [40]:
test_df['taxi_amount'] = (test_df['passenger_count'] // 4) +1

In [41]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   passenger_count  9914 non-null   int64  
 1   log_hour         9914 non-null   int64  
 2   log_min          9914 non-null   int64  
 3   log_year         9914 non-null   int64  
 4   log_month        9914 non-null   int64  
 5   log_day          9914 non-null   int64  
 6   7to15            9914 non-null   int64  
 7   16to19           9914 non-null   int64  
 8   20to6            9914 non-null   int64  
 9   dayofweek        9914 non-null   object 
 10  weekday          9914 non-null   int64  
 11  is_extracharge   9914 non-null   int64  
 12  dist             9914 non-null   float64
 13  taxi_amount      9914 non-null   int64  
dtypes: float64(1), int64(12), object(1)
memory usage: 1.1+ MB


In [42]:
test_df.to_csv("for_test.csv",index=False)